In [ ]:
import io

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

import json
# from pdfminer.converter import TextConverter
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfpage import PDFPage

In [ ]:
from bs4 import BeautifulSoup
import os
import re
import requests
import string
import numpy as np
import pandas as pd
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [ ]:
response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': 'https://www.al-akhbar.com/Editions',
    },
    )
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
container = soup.find('div', {'class': 'l-content row archive-issues'})
container = container.find('div', id='archive-days-wrap')

In [ ]:
dic = {}

for a_tag in container.find_all('a', class_='day'):
    data_id = a_tag.get('data-id')
    link = a_tag.get('href')
    url = 'https://www.al-akhbar.com' + link + '/PDF'
    data_name = a_tag.get('data-oldtitle')
    date = link[10:].replace('/','-')
    dic[data_id] = [data_name,url,date]

In [ ]:
df = pd.DataFrame(data=dic).T
df.columns = ['issue', 'url', 'date']
df = df[(df['date'] > '2010-01-01') & (df['date'] < '2020-01-01')]

In [ ]:

def get_pdf(url):
    
    retries = Retry(total=5,
                status_forcelist=[429, 500, 502, 503, 504])
    response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': '378d2b00-d774-4869-bf97-325ac3d1de22',
        'url': url,
    },
    )
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        container = soup.find('div', {'class': 'content'})
        pdf_link = container.find('a').get('href')
        pdf_link = 'https://www.al-akhbar.com' + pdf_link
        print('tamam')
        return pdf_link
    
    except:
        return None

In [ ]:
mask = (df.pdf_link.isna())
df2 = df.loc[mask]
df2['pdf_link'] = df2['url'].apply(get_pdf)
df2.to_csv('new.csv')

In [ ]:
mask3 = (df2.pdf_link.isna())
df3 = df2.loc[mask3]
df3

In [ ]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3.to_csv('new2.csv')

In [ ]:
df3 = pd.read_csv('new3.csv')
df3

In [ ]:
df3['pdf_link'] = df3['url'].apply(get_pdf)
df3

In [ ]:
df = pd.read_csv('pdf_df.csv')
df

In [ ]:
df1 = pd.read_csv('new.csv')
df2 = pd.read_csv('new2.csv')
df3 = pd.read_csv('new3.csv')

In [ ]:
df

In [ ]:
df.pdf_link.isna().value_counts()

In [ ]:
full_df = pd.concat([df1,df2,df3], ignore_index=True)
full_df.drop(columns=['Unnamed: 0'], inplace= True)

In [ ]:
full_df.drop_duplicates(inplace=True)

In [ ]:
full_df.pdf_link.isna().value_counts()

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = full_df['pdf_link']

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df['url'].apply(get_pdf)

In [ ]:
df.pdf_link.isna().value_counts()

In [ ]:
df.loc[df['pdf_link'].isna(), 'pdf_link'] = df.loc[df['pdf_link'].isna(), 'url'].apply(get_pdf)

In [ ]:
df.to_csv('Full_df.csv')

In [ ]:
df = pd.read_csv('Full_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

In [ ]:
os.chdir('/home/noahdarwich/code/NoahDarwich/Leb_analysis/Notebooks/')

In [ ]:
os.getcwd()

In [ ]:
def save_pdfs(df):
    
    unsaved = []
    df = df.dropna()
    for i, j in df.iterrows():
        
        name = j['date']+'.pdf'
        url = j['pdf_link']
        
        response = requests.get(url)
        
        try:
            os.chdir('/mnt/d')
            with open(name, 'wb') as file:
                file.write(response.content)
                print('tamam')
        except:
            unsaved.append(name)
            print('No')
    return unsaved

In [ ]:
unsaved = save_pdfs(df)

In [ ]:
import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
name = df.date[0]+'.pdf'
link = df.pdf_link[0]
response = requests.get(link)

In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-61adff20f07d.json', scopes=['https://www.googleapis.com/auth/drive'])

service = build('drive', 'v3', credentials=creds)

#file_metadata = {'name': name}



#file = service.files().create(body=file_metadata, media_body=media,
#                                    fields='id').execute()
#print(F'File ID: {file.get("id")}')

In [ ]:
folder_metadata = {
    'name': 'Al_Akhbar_2010_2019',
    'mimeType': 'application/vnd.google-apps.folder'
}
folder = service.files().create(body=folder_metadata, fields='id').execute()
folder_id = folder.get('id')
folder_id

In [ ]:
file = tempfile.NamedTemporaryFile()
file.write(response.content)
file.seek(0)

file_metadata = {'name': name,'parents':['1Le84wtKcUo5wP3bcBluZRmMcZnFsBNrk']}
media = MediaFileUpload(file.name,
                        mimetype='application/pdf')

file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

In [ ]:
# def save_article_to_drive(article_text, file_name):
#     try:
#       # authenticate with Google Drive
#       credentials = Credentials.from_authorized_user_info()
#       service = build('drive', 'v3', credentials=credentials)

#       # create the file on Google Drive
#       file_metadata = {'name': file_name, 'mimeType': 'text/plain'}
#       media = MediaIoBaseUpload(io.StringIO(article_text), mimetype='text/plain',chunksize=1024*1024, resumable=True)
#       file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
#       print(f'File ID: {file.get(/id\)}')
      
#     except HttpError as error:
#         print(f'An error occurred: {error}')
#         file = None
#     return file

In [ ]:
# scrape the website for PDF links

# download each PDF and save it to Google Drive

text_ls = [] 
for link in pdf_links:
    # download the PDF
    pdf_url = link['href']
    
    pdf_content = pdf_response.content
    # extract the text from the PDF
    text = extract_text_from_pdf(pdf_content)
    text_ls.append(text)
    
print(soup)

In [ ]:
# def extract_text_from_pdf(pdf_content):
#   resource_manager = PDFResourceManager()
#   fake_file_handle = io.StringIO()
#   converter = TextConverter(resource_manager, fake_file_handle)
#   page_interpreter = PDFPageInterpreter(resource_manager, converter)

#   with io.BytesIO(pdf_content) as fh:
#     for page in PDFPage.get_pages(fh,
#                                   caching=True,
#                                   check_extractable=True):
#       page_interpreter.process_page(page)

#     text = fake_file_handle.getvalue()

#   # close open handles
#   converter.close()
#   fake_file_handle.close()

#   return text

In [ ]:
# "# search the text for keywords
# "keywords = ['keyword1', 'keyword2', 'keyword3']
# "for keyword in keywords:
# "  if re.search(keyword, text, re.IGNORECASE):
# "    # keyword found, save the article
# "    file_name = f'{keyword}_article.txt'
# "    save_article_to_drive(text, file_name)
# "    break"